In [5]:
import pandas as pd

In [6]:
movies_details_df = pd.read_json('IMDB_movie_details.json', lines=True)
movies_details_df

,movie_id,plot_summary,duration,genre,rating,release_date,plot_synopsis
0,tt0105112,"Former CIA analyst, Jack Ryan is in England wi...",1h 57min,"[Action, Thriller]",6.9,1992-06-05,"Jack Ryan (Ford) is on a ""working vacation"" in..."
1,tt1204975,"Billy (Michael Douglas), Paddy (Robert De Niro...",1h 45min,[Comedy],6.6,2013-11-01,Four boys around the age of 10 are friends in ...
2,tt0243655,"The setting is Camp Firewood, the year 1981. I...",1h 37min,"[Comedy, Romance]",6.7,2002-04-11,
3,tt0040897,"Fred C. Dobbs and Bob Curtin, both down on the...",2h 6min,"[Adventure, Drama, Western]",8.3,1948-01-24,Fred Dobbs (Humphrey Bogart) and Bob Curtin (T...
4,tt0126886,Tracy Flick is running unopposed for this year...,1h 43min,"[Comedy, Drama, Romance]",7.3,1999-05-07,Jim McAllister (Matthew Broderick) is a much-a...
...,...,...,...,...,...,...,...
1567,tt0289879,Evan Treborn grows up in a small town with his...,1h 53min,"[Sci-Fi, Thriller]",7.7,2004-01-23,"In the year 1998, Evan Treborn (Ashton Kutcher..."
1568,tt1723811,Brandon is a 30-something man living in New Yo...,1h 41min,[Drama],7.2,2012-01-13,"Brandon (Michael Fassbender) is a successful, ..."
1569,tt5013056,Evacuation of Allied soldiers from the British...,1h 46min,"[Action, Drama, History]",8.1,2017-07-21,The film alternates between three different pe...
1570,tt0104014/,"For a while now, beautiful 24-year-old Diana B...",1h 33min,"[Comedy, Drama]",5.3,1992-02-21,


In [3]:
reviews_df = pd.read_json('IMDB_reviews.json', lines=True)
reviews_df

,review_date,movie_id,user_id,is_spoiler,review_text,rating,review_summary
0,10 February 2006,tt0111161,ur1898687,True,"In its Oscar year, Shawshank Redemption (writt...",10,A classic piece of unforgettable film-making.
1,6 September 2000,tt0111161,ur0842118,True,The Shawshank Redemption is without a doubt on...,10,Simply amazing. The best film of the 90's.
2,3 August 2001,tt0111161,ur1285640,True,I believe that this film is the best story eve...,8,The best story ever told on film
3,1 September 2002,tt0111161,ur1003471,True,"**Yes, there are SPOILERS here**This film has ...",10,Busy dying or busy living?
4,20 May 2004,tt0111161,ur0226855,True,At the heart of this extraordinary movie is a ...,8,"Great story, wondrously told and acted"
...,...,...,...,...,...,...,...
573908,8 August 1999,tt0139239,ur0100166,False,"Go is wise, fast and pure entertainment. Assem...",10,The best teen movie of the nineties
573909,31 July 1999,tt0139239,ur0021767,False,"Well, what shall I say. this one´s fun at any ...",9,Go - see the movie
573910,20 July 1999,tt0139239,ur0392750,False,"Go is the best movie I have ever seen, and I'v...",10,It's the best movie I've ever seen
573911,11 June 1999,tt0139239,ur0349105,False,Call this 1999 teenage version of Pulp Fiction...,3,Haven't we seen this before?


# Presentation of the Dataset

In [27]:
#TODO

# Preprocessing

## Tokenizer with regular expressions

In [6]:
!pip install nltk

Defaulting to user installation because normal site-packages is not writeable


In [7]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /home/lyes/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [8]:
imdb_spoilers_list = list(movies_details_df['plot_synopsis']) \
                    + list(movies_details_df['plot_summary']) \
                    + list(reviews_df['review_summary']) \
                    + list(reviews_df['review_text'])

imdb_spoilers_strings = " ".join(imdb_spoilers_list)

In [12]:
from nltk.tokenize import word_tokenize, sent_tokenize

# Have to reduce cause the len of `imdb_spoilers_strings` is 872 920 167 which way too big for my PC to tokenize (I might use Spark)
imdb_spoilers_strings_reduced = imdb_spoilers_strings[:1000000]
imdb_spoilers_tokens = word_tokenize(imdb_spoilers_strings_reduced) 

## Tokenizer with BPE

In [25]:
!pip install tiktoken

Defaulting to user installation because normal site-packages is not writeable


In [19]:
import tiktoken

In [20]:
enc = tiktoken.encoding_for_model("gpt-4")
imdb_spoilers_tokens_bpe = enc.encode(imdb_spoilers_strings_reduced)

## Normalisation

### Deleting the stopwords && put in lowercase the words

In [21]:
from nltk.corpus import stopwords
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /home/lyes/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [22]:
stop_words = set(stopwords.words('english'))
imdb_spoilers_tokens = [w.lower() for w in imdb_spoilers_tokens if not w.lower() in stop_words]

### Lemmatization : Changes the word to their inflected form (sleeping -> sleep)

In [23]:
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to /home/lyes/nltk_data...


True

In [24]:
lemmatizer = WordNetLemmatizer()
imdb_spoilers_tokens = [lemmatizer.lemmatize(w) for w in imdb_spoilers_tokens]

# Descriptive statistics

# Training on several predictive models

##### This Bayesian naive models is only about the `movies_details_df` it does not include the `reviews_df`

Which part contains spoilers ?

1. Plot summary : Does not contain spoiler
2. Plot synopsis: Contains Spoiler

In [43]:
# Cleaning the plot_synopsis column

plot_synopsis = movies_details_df.loc[movies_details_df['plot_synopsis'] != '', 'plot_synopsis']

In [44]:
# Outputs the length of the 2 category

no_spoil_length = len(movies_details_df['plot_summary'])
spoil_length = len(plot_synopsis)

In [53]:
# Display the appropriate dataFrame for our Bayesian naive model : Text - Is Spoil

all_texts = pd.concat([movies_details_df['plot_summary'], plot_synopsis])
spoil_categories = ['Not a Spoil'] * no_spoil_length + ['Spoil'] * spoil_length
is_spoiler_df = pd.DataFrame({'Text': all_texts, 'is_spoil': spoil_categories})

In [54]:
is_spoiler_df

,Text,is_spoil
0,"Former CIA analyst, Jack Ryan is in England wi...",Not a Spoil
1,"Billy (Michael Douglas), Paddy (Robert De Niro...",Not a Spoil
2,"The setting is Camp Firewood, the year 1981. I...",Not a Spoil
3,"Fred C. Dobbs and Bob Curtin, both down on the...",Not a Spoil
4,Tracy Flick is running unopposed for this year...,Not a Spoil
...,...,...
1563,The film opens with a homeless man (Bud Cort) ...,Spoil
1565,Will Freeman (Hugh Grant) is a 38-year-old bac...,Spoil
1567,"In the year 1998, Evan Treborn (Ashton Kutcher...",Spoil
1568,"Brandon (Michael Fassbender) is a successful, ...",Spoil
